In [1]:
# Imports
import pandas as pd

    1) Develop a prediction model to classify the customers as good or bad;
    2) Cluster the customers into various groups;
    3) Provide some ideas on how frequent pattern mining could be utilized to uncover some patterns in the data and/or to enhance the classification;

In [9]:
data = pd.read_csv('./dataset/project_data.csv', delimiter=';')
data

,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,...,X12,X13,X14,X15,X16,X17,X18,X19,X20,Y
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,A14,12,A32,A42,1736,A61,A74,3,A92,A101,...,A121,31,A143,A152,1,A172,1,A191,A201,1
996,A11,30,A32,A41,3857,A61,A73,4,A91,A101,...,A122,40,A143,A152,1,A174,1,A192,A201,1
997,A14,12,A32,A43,804,A61,A75,4,A93,A101,...,A123,38,A143,A152,1,A173,1,A191,A201,1
998,A11,45,A32,A43,1845,A61,A73,4,A93,A101,...,A124,23,A143,A153,1,A173,1,A192,A201,2


### Steps
1. Rename columns names for explicit visual understanding of features  

In [12]:
def rename_columns(dataframe: pd.DataFrame):
    target_columns = {
        'X01': 'account_status',
        'X06': 'account_savings',
        'X02': 'credit_duration',
        'X03': 'credit_history',
        'X04': 'credit_purpose',
        'X05': 'credit_amount',
        'X07': 'employment_status',
        'X17': 'employment_description',
        'X08': 'income_installment_rate',
        'X09': 'gender_status',
        'X10': 'credit_guarantors',
        'X11': 'residence',
        'X12': 'owned_property',
        'X13': 'age',
        'X14': 'installment_plans',
        'X15': 'accomondation_type',
        'X16': 'credit_existing_number',
        'X18': 'liable_maintain',
        'X19': 'phone_number',
        'X20': 'foreign_worker',
        'Y': 'y'
    }
    return dataframe.rename(columns=target_columns)    

In [13]:
data_1 = rename_columns(data)
data_1

,account_status,credit_duration,credit_history,credit_purpose,credit_amount,account_savings,employment_status,income_installment_rate,gender_status,credit_guarantors,...,owned_property,age,installment_plans,accomondation_type,credit_existing_number,employment_description,liable_maintain,phone_number,foreign_worker,y
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,A14,12,A32,A42,1736,A61,A74,3,A92,A101,...,A121,31,A143,A152,1,A172,1,A191,A201,1
996,A11,30,A32,A41,3857,A61,A73,4,A91,A101,...,A122,40,A143,A152,1,A174,1,A192,A201,1
997,A14,12,A32,A43,804,A61,A75,4,A93,A101,...,A123,38,A143,A152,1,A173,1,A191,A201,1
998,A11,45,A32,A43,1845,A61,A73,4,A93,A101,...,A124,23,A143,A153,1,A173,1,A192,A201,2


In [25]:
# data_1['age'].mean()
data_1.groupby('age')['credit_amount'].describe()

,count,mean,std,min,25%,50%,75%,max
age,,,,,,,,
19,2.0,1481.500000,704.985461,983.0,1232.25,1481.5,1730.75,1980.0
20,14.0,2139.785714,1182.783951,585.0,1289.75,2003.0,2926.50,4675.0
21,14.0,3799.357143,4269.882625,886.0,1166.75,2278.5,3338.50,15653.0
22,27.0,2109.592593,1326.284900,276.0,1294.50,1808.0,2960.00,5951.0
23,48.0,2993.354167,3342.965434,392.0,1212.00,1890.5,3509.25,15672.0
24,44.0,3502.363636,2778.767253,433.0,1552.00,2268.5,5007.00,11590.0
25,41.0,3165.439024,2690.408771,685.0,1295.00,2327.0,3972.00,14421.0
26,50.0,2927.940000,2255.365018,590.0,1333.75,2424.5,4094.75,10974.0
27,51.0,3390.588235,2950.126392,343.0,1266.00,2463.0,3935.00,14027.0
